# uproot 3 features

This notebook is primarily about what you can do in uproot 3 that you couldn't do in uproot 2. See the [main tutorial](https://mybinder.org/v2/gh/scikit-hep/uproot/master?urlpath=lab/tree/binder%2Ftutorial.ipynb) for a basic introduction to uproot.

The main differences are:

   * **more modularization:** uproot is concerned purely with file I/O; functions for interacting with histograms and physics objects like TLorentzVectors have been moved to [uproot-methods](https://github.com/scikit-hep/uproot-methods). Nested data (jagged arrays) have moved to the new [awkward-array](https://github.com/scikit-hep/awkward-array) library. Also, uproot no longer has its own implementation of dict-like caches— instead, we use the third-party [cachetools](https://cachetools.readthedocs.io/en/latest/) library. As a user, this means that you can use whichever combination of versions you need: e.g. bleeding-edge object methods with a stable file I/O.
   * **jagged and object array operations:** now when you read data with structure or class definitions, you can perform Numpy-like operations on them. More on that below.
   * **writing ROOT files:** basic support for writing ROOT files has begun, with more on the way.

Let's get started!

This notebook has uproot 3 installed with one example file:

```
wget http://scikit-hep.org/uproot/examples/HZZ-objects.root
```

In [1]:
import math
import numpy
import uproot
events = uproot.open("HZZ-objects.root")["events"]

## Object arrays

Since uproot 2, you could read TTree branches describing class objects (defined by ROOT's streamers). This includes STL collections, TVectors, and even histograms— if you should want to store histograms in a TTree. However, it also meant leaving the high-speed Numpy world for slow pure-Python code. In uproot 3, fixed-size objects are interpreted as `awkward.ObjectArrays`, which do most operations in a vectorized way across the fields of the data.

In [2]:
MET = events.array("MET")
MET

<ObjectArrayMethods [TVector2(5.9128, 2.5636) TVector2(24.765, -16.349) TVector2(-25.785, 16.237) ... TVector2(18.102, 50.291) TVector2(79.875, -52.351) TVector2(19.714, -3.5954)] at 0x7656000b0908>

The array above appears to contain `TVector2` objects. However, it only creates `TVector2` objects on demand (such as in the print-out). What we have actually loaded is the individual fields (`fX` and `fY`) as separate Numpy arrays.

In [3]:
MET.columns

[' fBits', ' fUniqueID', 'fX', 'fY']

(The `fBits` and `fUniqueID` columns exist in the data, but they're not interesting for physics. This is hinted by the fact that they begin with a space, a non-identifier character.)

In [4]:
MET["fX"]

array([  5.91277122,  24.76520348, -25.78508759, ...,  18.10164642,
        79.87519073,  19.71374893])

In [5]:
MET["fY"]

array([  2.5636332 , -16.34910965,  16.23713112, ...,  50.29071808,
       -52.35145187,  -3.59541821])

Although you can pull out any element as a `TVector2` Python object (creating it on demand),

In [6]:
MET[2], math.sqrt(MET[2].x**2 + MET[2].y**2)

(TVector2(-25.785, 16.237), 30.471546871754967)

or (gasp!) iterate over them in a Python for loop,

In [7]:
for i, met in enumerate(MET):
    print(met, math.sqrt(met.x**2 + met.y**2))
    if i > 10:
        break

TVector2(5.9128, 2.5636) 6.444616261735072
TVector2(24.765, -16.349) 29.67505163503274
TVector2(-25.785, 16.237) 30.471546871754967
TVector2(8.6199, -22.787) 24.362457116812326
TVector2(5.3931, -1.3101) 5.5499715598881885
TVector2(-3.7595, -19.417) 19.777622472715098
TVector2(23.962, -9.0492) 25.61389850143991
TVector2(-57.533, -20.488) 61.072343275467276
TVector2(42.416, -94.351) 103.44669393597131
TVector2(-1.9145, -23.963) 24.039388019581505
TVector2(19.71, 4.6455) 20.250114726294257
TVector2(-35.538, -14.754) 38.47893782676562


it's much faster to do operations across the whole array.

In [8]:
numpy.sqrt(MET.x**2 + MET.y**2)

array([ 6.44461626, 29.67505164, 30.47154687, ..., 53.4492837 ,
       95.50246389, 20.03893533])

Physics objects like `TVector2` have many methods, which are computed in Python if applied to a Python object but computed in Numpy if applied to a whole array.

In [9]:
MET[2].mag

30.471546871754967

In [10]:
MET.mag

array([ 6.44461626, 29.67505164, 30.47154687, ..., 53.4492837 ,
       95.50246389, 20.03893533])

In [11]:
MET[2].phi

2.5796134389921948

In [12]:
MET.phi

array([ 0.40911176, -0.58348763,  2.57961344, ...,  1.22529377,
       -0.58017296, -0.1803985 ])

Numpy operations apply elementwise to this array of `TVector2`, just as they would to Numpy arrays. Addition and multiplication are handled in physically meaningful ways.

In [13]:
MET + MET     # or numpy.add(MET, MET)

<ObjectArrayMethods [TVector2(11.826, 5.1273) TVector2(49.53, -32.698) TVector2(-51.57, 32.474) ... TVector2(36.203, 100.58) TVector2(159.75, -104.7) TVector2(39.427, -7.1908)] at 0x765600045208>

In [14]:
MET * 2       # same output

<ObjectArrayMethods [TVector2(11.826, 5.1273) TVector2(49.53, -32.698) TVector2(-51.57, 32.474) ... TVector2(36.203, 100.58) TVector2(159.75, -104.7) TVector2(39.427, -7.1908)] at 0x765600045780>

In [15]:
try:
    MET + 2
except TypeError:
    print("You can't add scalars to vectors (or multiply vectors to vectors).")

You can't add scalars to vectors (or multiply vectors to vectors).


In [16]:
MET.dot(MET)

array([  41.53307876,  880.60868954,  928.51516876, ..., 2856.8259281 ,
       9120.72060822,  401.55892909])

In [17]:
MET.mag2

array([  41.53307876,  880.60868954,  928.51516876, ..., 2856.8259281 ,
       9120.72060822,  401.55892909])

In [18]:
MET.delta_phi(MET)

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
MET.isparallel(MET)

array([ True,  True,  True, ...,  True,  True,  True])

## Jagged array operations

Just as in uproot 2, array-valued and `std::vector`-valued branches are presented as `JaggedArrays`. Unlike uproot 2, these are now being developed in an external library, awkward-array, with a rich set of operations that extend Numpy's built-in rules for broadcasting and indexing arrays.

In [20]:
muoniso = events.array("muoniso")
muoniso

<JaggedArray [[4.200153350830078 2.1510612964630127] [2.188047409057617] [1.412821650505066 3.3835041522979736] ... [3.7629451751708984] [0.550810694694519] [0.0]] at 0x7656000457b8>

Just as with object arrays, this is not a array of thousands of one- and two-element arrays; this is efficiently stored as contiguous arrays that generate subarrays on demand.

In [21]:
muoniso[2]

array([1.4128217, 3.3835042], dtype=float32)

In [22]:
for i, event_muoniso in enumerate(muoniso):
    for particle_muoniso in event_muoniso:
        print(particle_muoniso, end="     \t")
    print()
    if i > 10:
        break

4.2001534     	2.1510613     	
2.1880474     	
1.4128217     	3.3835042     	
2.7284882     	0.5522966     	
0.0     	0.8563976     	
0.0     	1.4929442     	
0.6231756     	0.0     	
2.4025257     	
0.0     	0.0     	
0.0     	1.7698176     	
2.0015755     	0.6041591     	
0.0     	0.76338214     	


In [23]:
muoniso.content

array([4.2001534, 2.1510613, 2.1880474, ..., 3.7629452, 0.5508107,
       0.       ], dtype=float32)

In [24]:
muoniso.counts

array([2, 1, 2, ..., 1, 1, 1])

What's new is that you can manipulate jagged arrays without resorting to for loops. This makes for more succinct code (a few characters, rather than the indented body of a for loop), but it's also much faster because it is implemented in Numpy (vectorized; contiguous memory access).

In the following, we multiply each muon isolation variable by the muon charge (just an example, not physically meaningful) and maintain the structure of which muon belongs to which event.

In [25]:
muoniso * events.array("muonq")

<JaggedArray [[4.200153350830078 -2.1510612964630127] [2.188047409057617] [1.412821650505066 -3.3835041522979736] ... [-3.7629451751708984] [-0.550810694694519] [-0.0]] at 0x7656000b6470>

Naturally, the jagged structure of the two arrays must match.

In [26]:
try:
    muoniso + events.array("electroniso")
except ValueError:
    print("Not all events have the same number of electrons as muons (in fact, most don't).")

Not all events have the same number of electrons as muons (in fact, most don't).


But if you operate on a jagged array and a flat array, you can "broadcast" each event's value to all associated particles' values. Here's a simple example to illustrate that:

In [27]:
import awkward
per_particle = awkward.JaggedArray.fromiter([[1, 2, 3], [], [4, 5]])
per_event = numpy.array([100, 200, 300])

per_particle + per_event

<JaggedArray [[101 102 103] [] [304 305]] at 0x7656000b6a90>

100 was added to all three of the particles in the first event, 200 was added to nothing because there are no particles in the second event, and 300 was added to all particles in the third event.

As a physically meaningful example, you might want to take the phi difference between each jet in an event with the event's MET.

In [28]:
events.array("jetp4").phi - events.array("MET").phi

<JaggedArray [[] [3.2527025700938905] [] ... [-2.8956145189632685] [3.448950466763478 -1.5021942373563388] []] at 0x7656000b6ef0>

Or use the built-in `delta_phi` to correctly handle wrap-around.

In [29]:
events.array("jetp4").delta_phi(events.array("MET"))

<JaggedArray [[] [-3.0304827370856957] [] ... [-2.8956145189632685] [-2.8342348404161086 -1.5021942373563388] []] at 0x7656000b6d68>

All but the last example involved jagged arrays of numbers. This last one combined jagged arrays with object arrays, as `delta_phi` is an object method. That's worth a new section.

## Jagged object arrays

The idea of the awkward-array library is that these special array features should be compositional: you can have jagged arrays of jagged arrays and all of the above can have object-like methods.

That is most clearly demonstrated with a jagged array of `TLorentzVectors`.

In [30]:
muonp4 = events.array("muonp4")
muonp4

<JaggedArrayMethods [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [TLorentzVector(48.988, -21.723, 11.168, 54.74) TLorentzVector(0.82757, 29.801, 36.965, 47.489)] ... [TLorentzVector(-29.757, -15.304, -52.664, 62.395)] [TLorentzVector(1.1419, 63.61, 162.18, 174.21)] [TLorentzVector(23.913, -35.665, 54.719, 69.556)]] at 0x765600069828>

Derived quantities like `pt` and `eta` have the same jagged structure as the `TLorentzVectors`.

In [31]:
muonp4.pt

<JaggedArray [[54.16810703140204 37.74415265978988] [24.41791248135961] [53.58826697278532 29.811997139120674] ... [33.461536523629654] [63.619817705512375] [42.93994828083644]] at 0x7656000b69e8>

In [32]:
muonp4.eta

<JaggedArray [[-0.15009261562098436 -0.29527552510823807] [0.7538136927232744] [0.20692921428668967 1.0412952985056445] ... [-1.235046871298247] [1.665331082845152] [1.0626988631804193]] at 0x7656000b6fd0>

Note that the jagged array of `TLorentzVector` has type `JaggedArrayMethods` but the jagged arrays of numbers have type `JaggedArray`. That is so that you know which jagged arrays have been imbued with physics methods. For instance, it wouldn't make any sense to take the `pt()` of a number:

In [33]:
try:
    muonp4.pt.pt
except AttributeError:
    print("muonp4.pt is a plain JaggedArray; it doesn't have methods like .pt")

muonp4.pt is a plain JaggedArray; it doesn't have methods like .pt


The same is true for `ObjectArray` (returns objects when indexed, but doesn't have physics methods) and `ObjectArrayMethods` (returns objects and has physics methods) and any other awkward array type. If you want to check types with `isinstance`, be careful— these types are generated dynamically and one `JaggedArrayMethods` is not necessarily the same type as another `JaggedArrayMethods`. Check the physics-specific supertype.

In [34]:
import uproot_methods

# this checks to see if muonp4 has TLorentzVector methods that can be applied across arrays
isinstance(muonp4, uproot_methods.classes.TLorentzVector.ArrayMethods)

True

In [35]:
# the same is true for flat arrays with TLorentzVector methods
isinstance(muonp4[0], uproot_methods.classes.TLorentzVector.ArrayMethods)

True

### Masking

Just as with Numpy arrays, we can apply masks to select events of interest.

In [36]:
mask = (muonp4.counts >= 2)
mask

array([ True, False,  True, ..., False, False, False])

In [37]:
muonp4[mask]

<JaggedArrayMethods [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(48.988, -21.723, 11.168, 54.74) TLorentzVector(0.82757, 29.801, 36.965, 47.489)] [TLorentzVector(22.088, -85.835, 403.85, 413.46) TLorentzVector(76.692, -13.956, 335.09, 344.04)] ... [TLorentzVector(53.006, -24.486, 13.952, 60.032) TLorentzVector(-30.209, 19.269, 18.661, 40.399)] [TLorentzVector(55.72, 26.37, -24.588, 66.368) TLorentzVector(-26.914, -9.8128, -0.38995, 28.65)] [TLorentzVector(34.507, 28.84, -150.66, 157.23) TLorentzVector(-31.568, -10.424, -111.26, 116.13)]] at 0x765600069b00>

### Multidimensional indexes

Also like Numpy arrays, we can apply multidimensional indexes to pick out, say, the first and second muon in events with two muons.

In [38]:
muon0 = muonp4[mask, 0]
muon0

<ObjectArrayMethods [TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(48.988, -21.723, 11.168, 54.74) TLorentzVector(22.088, -85.835, 403.85, 413.46) ... TLorentzVector(53.006, -24.486, 13.952, 60.032) TLorentzVector(55.72, 26.37, -24.588, 66.368) TLorentzVector(34.507, 28.84, -150.66, 157.23)] at 0x7656000730f0>

In [39]:
muon1 = muonp4[mask, 1]
muon1

<ObjectArrayMethods [TLorentzVector(37.738, 0.69347, -11.308, 39.402) TLorentzVector(0.82757, 29.801, 36.965, 47.489) TLorentzVector(76.692, -13.956, 335.09, 344.04) ... TLorentzVector(-30.209, 19.269, 18.661, 40.399) TLorentzVector(-26.914, -9.8128, -0.38995, 28.65) TLorentzVector(-31.568, -10.424, -111.26, 116.13)] at 0x765600073518>

Note that these are now non-jagged arrays. By asking for the N<sup>th</sup> in each event, we now have something that is one-per-event: non-jagged.

But they are object arrays with array methods. We can do things with them like compute masses.

In [40]:
(muon0 + muon1).mass

array([90.22779777, 74.74654928, 89.75736376, ..., 92.06495256,
       85.44384208, 75.96066262])

### Jagged mask

An ordinary Numpy mask selects events. To select particles, make a jagged array of booleans.

In [41]:
mask = (muonp4.pt > 40)
mask

<JaggedArray [[True False] [False] [True False] ... [False] [True] [True]] at 0x765600073ac8>

In [42]:
muonp4[mask]

<JaggedArrayMethods [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779)] [] [TLorentzVector(48.988, -21.723, 11.168, 54.74)] ... [] [TLorentzVector(1.1419, 63.61, 162.18, 174.21)] [TLorentzVector(23.913, -35.665, 54.719, 69.556)]] at 0x765600073c18>

### Jagged reduction

Calling a "reducer" like `.sum()`, `.min()`, or `.max()` on a Numpy array returns a scalar. Calling the same reducer on a jagged array returns a flat array: you get the sum/min/max per event.

In [43]:
abseta = abs(muonp4.eta)
abseta

<JaggedArray [[0.15009261562098436 0.29527552510823807] [0.7538136927232744] [0.20692921428668967 1.0412952985056445] ... [1.235046871298247] [1.665331082845152] [1.0626988631804193]] at 0x765600081128>

In [44]:
abseta.max()

array([0.29527553, 0.75381369, 1.0412953 , ..., 1.23504687, 1.66533108,
       1.06269886])

This can be useful for making event-level masks from particle data.

In [45]:
muonp4[abseta.max() < 1]    # events in which both muons are within |eta| < 1

<JaggedArrayMethods [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [TLorentzVector(45.171, 67.249, -89.696, 120.86) TLorentzVector(39.751, 25.404, 20.115, 51.285)] ... [TLorentzVector(53.006, -24.486, 13.952, 60.032) TLorentzVector(-30.209, 19.269, 18.661, 40.399)] [TLorentzVector(55.72, 26.37, -24.588, 66.368) TLorentzVector(-26.914, -9.8128, -0.38995, 28.65)] [TLorentzVector(-24.158, -35.032, -19.194, 46.683)]] at 0x765600081860>

### Jagged argmin/argmax

Sometimes you want to know variable `x` that is minimized/maximized over `y`. Numpy uses `argmin` and `argmax` with fancy indexing to provide that for flat arrays; we can now do the same for jagged arrays.

The `argmin` and `argmax` of a jagged array returns `[]` for empty events and `[i]` for non-empty events, where `i` is the index of the minimized/maximized item.

In [46]:
abseta.argmax()

<JaggedArray [[1] [0] [1] ... [0] [0] [0]] at 0x765600081a58>

Passing jagged indexes into the square brackets of a jagged array picks the subitems by index. Note that each event contains zero (`[]`) or one (`[TLorentzVector(...)]`) item.

In [47]:
muonp4[abseta.argmax()]

<JaggedArrayMethods [[TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [TLorentzVector(0.82757, 29.801, 36.965, 47.489)] ... [TLorentzVector(-29.757, -15.304, -52.664, 62.395)] [TLorentzVector(1.1419, 63.61, 162.18, 174.21)] [TLorentzVector(23.913, -35.665, 54.719, 69.556)]] at 0x765600006128>

This would be more clear if we look at plain numbers.

In [48]:
muonp4[abseta.argmax()].pt

<JaggedArray [[37.74415265978988] [24.41791248135961] [29.811997139120674] ... [33.461536523629654] [63.619817705512375] [42.93994828083644]] at 0x765600081080>

To get a min/max for only the non-empty events, you can flatten the result, which effectively concatenates all subarrays: `[]` disappear and `[x]` become `x`. (Because of our data representation, this is merely discarding the structure information, so it's instantaneous.)

In [49]:
muonp4[abseta.argmax()].flatten()

<ObjectArrayMethods [TLorentzVector(37.738, 0.69347, -11.308, 39.402) TLorentzVector(-0.81646, -24.404, 20.2, 31.69) TLorentzVector(0.82757, 29.801, 36.965, 47.489) ... TLorentzVector(-29.757, -15.304, -52.664, 62.395) TLorentzVector(1.1419, 63.61, 162.18, 174.21) TLorentzVector(23.913, -35.665, 54.719, 69.556)] at 0x7656000063c8>

### Jagged fancy indexing

Although a major use of fancy indexing is to consume the result of an `argmin` or `argmax`, it's a general and useful technique.

In Numpy, you can pass an arbitrary sequence of indexes to an array to pick out just those elements, in the specified order. (Also useful for consuming the result of `argsort`.) If you pass such a sequence to a jagged array, you get an analogous result: selected events in the order you request them.

In [50]:
import awkward
simple = awkward.JaggedArray.fromiter([[0.0, 1.1, 2.2], [], [3.3, 4.4]])
simple[[2, 1, 2]]

<JaggedArray [[3.3 4.4] [] [3.3 4.4]] at 0x765600006b70>

But if you pass a jagged array of indexes to a jagged array, you select particles within each event.

In [51]:
simple[awkward.JaggedArray.fromiter([[2, 1, 2], [], [1, 1, 1, 0]])]

<JaggedArray [[2.2 1.1 2.2] [] [4.4 4.4 4.4 3.3]] at 0x765600006c50>

This is in exact analogy with the treatment of jagged masks.

### Jagged cross-join and pairs

Numpy-like extensions let you mask particles and events, select them by index, sum/min/max over them by event, apply transformations across arrays and jagged arrays while maintaining their structure, and broadcast per-event attributes to each particle. However, none of these let you form new candidates by combining particles.

For example, say you want to loop over all jets and all muons to search for leptoquarks.

In [52]:
jetp4, muonp4 = events.arrays(["jetp4", "muonp4"], outputtype=tuple)

In [53]:
leptoquarks = []
for i in range(len(events)):
    current = []
    for jet in jetp4[i]:
        for muon in muonp4[i]:
            current.append(jet + muon)
    leptoquarks.append(current)
    if i > 10:
        break

leptoquarks

[[],
 [TLorentzVector(-39.691, -4.5408, 19.305, 75.828)],
 [],
 [TLorentzVector(-49.607, 7.7361, 600.14, 643.81),
  TLorentzVector(4.9967, 79.615, 531.39, 574.39),
  TLorentzVector(58.695, -63.997, 495.51, 514.82),
  TLorentzVector(113.3, 7.8823, 426.76, 445.4),
  TLorentzVector(-6.7781, -76.515, 455.09, 473.54),
  TLorentzVector(47.825, -4.6358, 386.34, 404.13)],
 [TLorentzVector(49.051, -7.9853, -449.3, 488.45),
  TLorentzVector(43.631, -49.83, -339.49, 418.87),
  TLorentzVector(50.151, 28.017, -21.239, 200.11),
  TLorentzVector(44.731, -13.828, 88.572, 130.53)],
 [TLorentzVector(-37.099, 67.813, 36.758, 118.54),
  TLorentzVector(-52.121, -3.0363, 94.355, 127.32),
  TLorentzVector(36.966, 26.013, -9.3573, 76.646),
  TLorentzVector(21.944, -44.837, 48.239, 85.435)],
 [TLorentzVector(-69.527, 5.1032, -386.78, 411.54),
  TLorentzVector(-52.524, 43.206, -288.69, 319.26)],
 [TLorentzVector(62.803, -48.904, 386.53, 395.3)],
 [],
 [TLorentzVector(-29.129, 44.266, -14.775, 181.23),
  TLorent

It works, but it's slow because of the Python for loops and the Python objects. This operation of pairing every item in set `muons` with every item in set `jets` is known in databases as a "cross-join" or a "cartesian product."

Expanding two huge sets with a cross-join would use a lot more memory than iterating over pairs, but with jagged arrays, only events would be cross-joined. You might have a million events, but if the largest number of particles in any one event is `N` and `M`, the largest number of elements in the per-event cross-join is `N*M`. It does not scale with a million or a million squared. Operations on the expanded cross-join can be vectorized and much faster than operations in iterative loops.

In many situations, it's practical to cross-join particle data.

In [54]:
pairs = jetp4.cross(muonp4)

In [55]:
pairs.i0   # the first in each pair (._N is a shortcut for ["N"] for any integer N)

<JaggedArrayMethods [[] [TLorentzVector(-38.875, 19.863, -0.89494, 44.137)] [] ... [TLorentzVector(-3.7148, -37.202, 41.012, 55.951)] [TLorentzVector(-36.361, 10.174, 226.43, 229.58) TLorentzVector(-15.257, -27.175, 12.12, 33.92)] []] at 0x765600010550>

In [56]:
pairs.i1   # the second in each pair

<JaggedArrayMethods [[] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [] ... [TLorentzVector(-29.757, -15.304, -52.664, 62.395)] [TLorentzVector(1.1419, 63.61, 162.18, 174.21) TLorentzVector(1.1419, 63.61, 162.18, 174.21)] []] at 0x7656000179e8>

In [57]:
leptoquarks = pairs.i0 + pairs.i1
leptoquarks

<JaggedArrayMethods [[] [TLorentzVector(-39.691, -4.5408, 19.305, 75.828)] [] ... [TLorentzVector(-33.472, -52.506, -11.652, 118.35)] [TLorentzVector(-35.219, 73.783, 388.61, 403.79) TLorentzVector(-14.115, 36.434, 174.3, 208.13)] []] at 0x765600025160>

Now that we have this jagged array of leptoquark candidates, we can ask it questions, like, "What's your mass?"

In [58]:
leptoquarks.mass

<JaggedArray [[] [61.491172664049785] [] ... [99.96350949193341] [73.10963362904484 106.82646499328074] []] at 0x7656000250b8>

Sometimes you need to cross three (or more) collections.

In [59]:
photonp4 = events.array("photonp4")
jetp4.cross(muonp4).cross(photonp4)

<JaggedArray [[] [] [] ... [] [(TLorentzVector(-36.361, 10.174, 226.43, 229.58), TLorentzVector(1.1419, 63.61, 162.18, 174.21), TLorentzVector(-29.46, 8.7978, 189.47, 191.94)) (TLorentzVector(-15.257, -27.175, 12.12, 33.92), TLorentzVector(1.1419, 63.61, 162.18, 174.21), TLorentzVector(-29.46, 8.7978, 189.47, 191.94))] []] at 0x765600025ac8>

And sometimes you want to cross a collection with itself.

In [60]:
muonpairs = muonp4.cross(muonp4)
muonpairs

<JaggedArray [[(TLorentzVector(-52.899, -11.655, -8.1608, 54.779), TLorentzVector(-52.899, -11.655, -8.1608, 54.779)) (TLorentzVector(-52.899, -11.655, -8.1608, 54.779), TLorentzVector(37.738, 0.69347, -11.308, 39.402)) (TLorentzVector(37.738, 0.69347, -11.308, 39.402), TLorentzVector(-52.899, -11.655, -8.1608, 54.779)) (TLorentzVector(37.738, 0.69347, -11.308, 39.402), TLorentzVector(37.738, 0.69347, -11.308, 39.402))] [(TLorentzVector(-0.81646, -24.404, 20.2, 31.69), TLorentzVector(-0.81646, -24.404, 20.2, 31.69))] [(TLorentzVector(48.988, -21.723, 11.168, 54.74), TLorentzVector(48.988, -21.723, 11.168, 54.74)) (TLorentzVector(48.988, -21.723, 11.168, 54.74), TLorentzVector(0.82757, 29.801, 36.965, 47.489)) (TLorentzVector(0.82757, 29.801, 36.965, 47.489), TLorentzVector(48.988, -21.723, 11.168, 54.74)) (TLorentzVector(0.82757, 29.801, 36.965, 47.489), TLorentzVector(0.82757, 29.801, 36.965, 47.489))] ... [(TLorentzVector(-29.757, -15.304, -52.664, 62.395), TLorentzVector(-29.757, -1

In [61]:
(muonpairs.i0 + muonpairs.i1).mass

/usr/local/miniconda3/lib/python3.7/site-packages/uproot_methods-0.4.3-py3.7.egg/uproot_methods/classes/TLorentzVector.py:196: RuntimeWarning: invalid value encountered in sqrt
  return self._trymemo("mass", lambda self: self.awkward.numpy.sqrt(self.mag2))


<JaggedArray [[0.2111859748287381 90.22779776988638 90.22779776988638 0.21090494082084574] [0.210998480800626] [0.21392618221202328 74.74654928040661 74.74654928040661 0.2102757625673823] ... [0.2108764933349408] [0.1950119912345726] [0.20894448339535043]] at 0x76560002f2e8>

We get a warning because some of these pairs have a negative mass-squared (resulting in `nan` after the square root). That's because some of these pairs combine a muon with itself, which is twice the muon mass (`0.211` GeV), which is close enough to zero for some error to bring it below zero.

We want to only combine _distinct_ pairs of muons, so instead of `cross`, we call `distincts` (distinct pairs should not include a muon with itself).

In [62]:
muonpairs = muonp4.distincts()
muonpairs

<JaggedArray [[(TLorentzVector(-52.899, -11.655, -8.1608, 54.779), TLorentzVector(37.738, 0.69347, -11.308, 39.402))] [] [(TLorentzVector(48.988, -21.723, 11.168, 54.74), TLorentzVector(0.82757, 29.801, 36.965, 47.489))] ... [] [] []] at 0x76560002fe80>

In [63]:
(muonpairs.i0 + muonpairs.i1).mass

<JaggedArray [[90.22779776988638] [] [74.74654928040661] ... [] [] []] at 0x76560002ff98>

A general workflow would generally fit the pattern of expanding over a cross-join or pairs, computing something, and then collapsing the structure down with a reduction. For instance, we compute Z candidates by combining pairs of muons and computing their mass, then select the best candidate (highest mass as an example below).

In [64]:
zcandidates = muonpairs.i0 + muonpairs.i1
zcandidates[zcandidates.mass.argmax()]

<JaggedArrayMethods [[TLorentzVector(-15.162, -10.961, -19.468, 94.181)] [] [TLorentzVector(49.815, 8.0774, 48.133, 102.23)] ... [] [] []] at 0x7655f5abf588>

The `cross` and `pairs` functions are not expected to solve all possible problems. There may be other combinations of nested structures that you need to do your physics analysis— if you have an idea for another useful function, let me know (via GitHub issue).

### Attaching to tables

Finally, it can be useful to glue data together, for bookkeeping. This is the only operation that modifies an object in place, and keep in mind that it does not modify the original ROOT file. It modifies your array object in memory. (See below for writing to ROOT files.)

In [65]:
muonp4 = events.array("muonp4")
muonp4

<JaggedArrayMethods [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [TLorentzVector(48.988, -21.723, 11.168, 54.74) TLorentzVector(0.82757, 29.801, 36.965, 47.489)] ... [TLorentzVector(-29.757, -15.304, -52.664, 62.395)] [TLorentzVector(1.1419, 63.61, 162.18, 174.21)] [TLorentzVector(23.913, -35.665, 54.719, 69.556)]] at 0x7655f5ac9668>

This `muonp4` is presented as a jagged array of `TLorentzVectors` for your convenience, but it's actually an awkward-array `Table` of the `TLorentzVector` private members defined by the ROOT file's schema.

In [66]:
muonp4.columns

[' fBits', ' fUniqueID', ' fBits2', ' fUniqueID2', 'fX', 'fY', 'fZ', 'fE']

In [67]:
muonp4["fX"]

<JaggedArray [[-52.89945602416992 37.7377815246582] [-0.8164593577384949] [48.987831115722656 0.8275666832923889] ... [-29.756786346435547] [1.1418697834014893] [23.913206100463867]] at 0x7655f5abfc88>

In [68]:
muonp4.x       # this is a property for more Pythonic access than muonp4["fX"]

<JaggedArray [[-52.89945602416992 37.7377815246582] [-0.8164593577384949] [48.987831115722656 0.8275666832923889] ... [-29.756786346435547] [1.1418697834014893] [23.913206100463867]] at 0x7655f5abff28>

We can add any derived fields to the `Table`.

In [69]:
muonp4["pt"] = muonp4.pt

In [70]:
muonp4["pt"]   # fetch precomputed value

<JaggedArray [[54.16810703140204 37.74415265978988] [24.41791248135961] [53.58826697278532 29.811997139120674] ... [33.461536523629654] [63.619817705512375] [42.93994828083644]] at 0x7655f5ac9048>

In [71]:
muonp4.pt      # recompute value

<JaggedArray [[54.16810703140204 37.74415265978988] [24.41791248135961] [53.58826697278532 29.811997139120674] ... [33.461536523629654] [63.619817705512375] [42.93994828083644]] at 0x7655f5ac9940>

You can even change values in memory. For instance, suppose our muon need an energy correction (?!?).

In [72]:
muonp4["fE"] = muonp4["fE"] * 2

In [73]:
muonp4.mass

<JaggedArray [[94.88093433185503 68.24581955176036] [54.88956214774407] [94.81215444319224 82.25318077863159] ... [108.07163987013124] [301.73821995993745] [120.47494085196762]] at 0x7655f5ac9fd0>

(Woah— heavy muons.)

## Writing ROOT files

TBD